# QCTO - Workplace Module

### Project Title: Customer segmentation based on behaviour to predict potential profitable clients at application a
#### Done By: Vuyiswa Kubalasa

© ExploreAI 2024

---

## Table of Contents

<a href=#BC> Background Context</a>

<a href=#one>1. Importing Packages</a>

<a href=#two>2. Data Collection and Description</a>

<a href=#three>3. Loading Data </a>

<a href=#four>4. Data Cleaning and Filtering</a>

<a href=#five>5. Exploratory Data Analysis (EDA)</a>

<a href=#six>6. Modeling </a>

<a href=#seven>7. Evaluation and Validation</a>

<a href=#eight>8. Final Model</a>

<a href=#nine>9. Conclusion and Future Work</a>

<a href=#ten>10. References</a>

---
 <a id="BC"></a>
## **Background Context**
<a href=#cont>Back to Table of Contents</a>

Banks have a large cutsomer base with varying behavioural characteristics in terms of spend behaviour, revolving behaviour, age, disposal income, etc. Most banks focus predominantly on booking clients with a low probability of defaulting, rightful so given the risk involved in borrowing money. However, the primary purpose of banks lending money is to make profit and a few banks looking into whether the clients they are booking are likely to be profitable. We argue that a good balance of expected risk and expected profitability should be considered when booking clients. Profitability can be seen as the long-term value of a customer. Customer Lifetime Value (CLTV) is a metric that represents the total amount of money a customer is expected to spend in a business/ bank during the lifetime of the card. It's a crucial measure that helps businesses in strategizing their marketing efforts, resource allocation and product development.

At application we may not have enough historical information to determine the clients probability of being profitable, however similare to determining whether a client will default or nor, through a detailed segmentation of existing customers using their  historical behaviour we can identify customer segments likely to be highly profitable during the lifetime of having the credit card.

**Project objective**: The primarly goal of this project to predict customers profit score in order to identify high value clients (high profitability with low to medium risk of defaulting). Profit score is the probability of generating high profitability. The main questions we aim to answer is:
>  1) can we predict whether a customer is likely to be profitable at application \
2) what customers segments result in high profitability \
3) can we suggest incentives to increase profit score as a clients continues

At the end of this project, the business will gain a comprehensive understanding of how certain customer behaviours or segments influence profitability, the project will help to effectively allocate resources. Instead of spreading resources over all customers, but identifying expected profitability at application banks can focus various efforts based on profit scoring. For identified high value (profitability) clients, the bank can focus on offering personalized rewards to retain these profitable segments. For idenified low value (profitable) clients banks can adjust pricing and product development strategies to that maximize profits without losing valuable customers. Furthermore, the project will help in in forecasting expected profitability likelihood and making informed decisions about potential investments in customer acquisition and retention.

**Methodology Overview**: Here is a brief overiwes of the methods and technique we plan to use in the analysis. We will:
> 1) determine a profitability proxy from the utilisation and spend data,\
2) determine a profit score from the profit value, we can convert the probabilities to binary categorization 0 for low profitable client and 1 for high profitable client based on a subjective profitability threshold \
3) use clustering and other machine learning algorithms to create a customer segmentation model after some comprehensive analysis and \
4) use model to predict profit scores.

**Structure of the Notebook**

---

---
<a href=#one></a>
## **Importing Packages**
<a href=#cont>Back to Table of Contents</a>


In [42]:
import pandas as pd 
import numpy as np
import os  # get path of notebook 

---
<a href=#two></a>
## **Data Collection and Description**
<a href=#cont>Back to Table of Contents</a>

The source the data data, we conducted some web scraping for credit card datasets and examined some dataset from github repositories, this data although it does not have all the fields we hoping for to conduct a comprehensive analysis, it contains the necessarily fields in comparison to the other credit card datasets. The data was source from [kaggle](https://www.kaggle.com/code/shawkyelgendy/credit-card-cs-eda-and-ml/notebook), it consists of aggregated credit card data including the customers total spend and balance throughout the lifetime of having the card. There are 10127 entries and 23 columns in total, an indepth data dictionary can be found [here](https://github.com/VuyiswaK/Workplace_project/blob/main/README.md).


---
<a href=#three></a>
## **Loading Data**
<a href=#cont>Back to Table of Contents</a>


In [15]:
path_all = os.getcwd()
df = pd.read_csv(path_all + '\credit_card_data.csv')

In [16]:
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
0,768805383,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,...,12691.0,777,11914.0,1.335,1144,42,1.625,0.061,0.000093,0.99991
1,818770008,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,...,8256.0,864,7392.0,1.541,1291,33,3.714,0.105,0.000057,0.99994
2,713982108,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,...,3418.0,0,3418.0,2.594,1887,20,2.333,0.000,0.000021,0.99998
3,769911858,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,...,3313.0,2517,796.0,1.405,1171,20,2.333,0.760,0.000134,0.99987
4,709106358,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,...,4716.0,0,4716.0,2.175,816,28,2.500,0.000,0.000022,0.99998


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10127 entries, 0 to 10126
Data columns (total 23 columns):
 #   Column                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                              --------------  -----  
 0   CLIENTNUM                                                                                                                           10127 non-null  int64  
 1   Attrition_Flag                                                                                                                      10127 non-null  object 
 2   Customer_Age                                                                                                                        10127 non-null  int64  
 3   Gender                                                                           

---
<a href=#four></a>
## **Data Cleaning and Filtering**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Prepare the data for analysis by cleaning and filtering.
* **Details:** Include steps for handling missing values, removing outliers, correcting errors, and possibly reducing the data (filtering based on certain criteria or features).
---

#### Check for duplicates 

In [20]:
df_copy = df.copy()  # copy dataframe
df_copy.duplicated().sum()


0

#### Handle missing values

In [24]:
for column in df_copy:
    if df_copy[column].isnull().any():
        print('{0} has {1} null values'.format(column, df_copy[column].isnull().sum()))
        
# No missing data 

#### Remove outliers

In [ ]:
#check utilization and spend outliers ()

#### Correcting errors & reducing data

Some columns have long column names, we reduce these column names

In [27]:
df_copy.rename(columns={'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1':'Attrition_mon_1',
                  'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2':'Attrition_mon_2'},inplace=True)

There are some columns from the onset that are not necessarily to our preminilary analysis or building the predictive model, we will remove them before feature select. For instance we do not need the primary key CLIENTNUM

In [28]:
df_copy.drop(columns=['CLIENTNUM'],inplace=True)

#### Adding profitability columns

Profitability can be determine from fields such the utilisation and spend, but have not be explicity done here, thus we generate a profitability proxy using the existing columns and this will be our response variable. And a profit scoring field More than 1 and 0, perhaps 5. 

**Profitability value**

For interest income, interest rate is applied to the revolving balance, as we earn interest on the revolved balance. We will the current NCA max interest rate in south africa which is about $22,25\%$.
 
 > $ \text{Interest income} = \text{Tot_Revolving_Balance} \, \cdot \, \text{Interest rate}$
    
    
    
For interchange income, this is the income the bank earns upon transacting with the credit card, we assume all transactions are done directly with the card, we use an interchange rate of $1,48\%$. Interchange rate depends on the credit card association and is usually between $1\%$ and $3\%$

 > $ \text{Interchange income} = \text{Total_Trans_Amt} \, \cdot \, \text{Interchange rate}$
 
 
Note there are more consideration when computing profitability like monthly maintanence and credit facility fees, however we do not have access to the information and are merely computing a proxy for profitability based on spend and revolving behaviour as fees will more ore less be the same for clients.
    
    
We will also consider expected loss and treat the attrition_mon_1 column which describe the probability of churching as the probability of default PD. EAD is Exposure at default which in our case will be the product of avg_utilization_ration and the credit limit. LGD is the loss given at defulat in South Africa it is usually between  $50\%$ and $70\%$, we will take a lower value $40\%$ assume the data is from a first world country .

 > $ \text{Expected Loss} = \text{PD} \, \cdot \, \text{EAD} \, \cdot \, \text{LGD}$
 
 We will leave out cost, as we don't have access to cost of funding and expense data.

Thus our profitability becomes:

 > $ \text{Profitability} = \text{Interchange income} + \text{Interest income} - \text{Expected Loss}$


In [31]:
df_copy.columns

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'Attrition_mon_1', 'Attrition_mon_2'],
      dtype='object')

In [47]:
interest_income = df_copy.Total_Revolving_Bal*(0.2225/12) #monthly interest rate
interchange_income = df_copy.Total_Trans_Amt*0.0148
expected_loss = df.Avg_Utilization_Ratio*df_copy.Credit_Limit*0.4*df_copy.Attrition_mon_1
df_copy['Profit'] = interest_income + interchange_income - expected_loss

In [48]:
min(df_copy.Profit),max(df_copy.Profit),np.mean(df_copy.Profit)

(-948.0062154000001, 307.46289566910923, 43.6415803900549)

---
<a href=#five></a>
## **Exploratory Data Analysis (EDA)**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Explore and visualize the data to uncover patterns, trends, and relationships.
* **Details:** Use statistics and visualizations to explore the data. This may include histograms, box plots, scatter plots, and correlation matrices. Discuss any significant findings.
---


#### Summary statistics

In [23]:
df_copy.describe()

,CLIENTNUM,Customer_Age,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1,Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
count,1.012700e+04,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000,10127.000000
mean,7.391776e+08,46.325960,2.346203,35.928409,3.812580,2.341167,2.455317,8631.953698,1162.814061,7469.139637,0.759941,4404.086304,64.858695,0.712222,0.274894,0.159997,0.840003
std,3.690378e+07,8.016814,1.298908,7.986416,1.554408,1.010622,1.106225,9088.776650,814.987335,9090.685324,0.219207,3397.129254,23.472570,0.238086,0.275691,0.365301,0.365301
min,7.080821e+08,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000,0.000008,0.000420
25%,7.130368e+08,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2555.000000,359.000000,1324.500000,0.631000,2155.500000,45.000000,0.582000,0.023000,0.000099,0.999660
50%,7.179264e+08,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4549.000000,1276.000000,3474.000000,0.736000,3899.000000,67.000000,0.702000,0.176000,0.000181,0.999820
75%,7.731435e+08,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,11067.500000,1784.000000,9859.000000,0.859000,4741.000000,81.000000,0.818000,0.503000,0.000337,0.999900
max,8.283431e+08,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,18484.000000,139.000000,3.714000,0.999000,0.999580,0.999990


Highlight interesting insights, and potential outliers, see when mean or median imply potential outliers or min or max compared to averages

#### Outlier detection and fix

Outliers not visible in the initial data exploration

#### Data visualiztion

Highlight interesting patterns

#### Correlation analysis 

Highlight interesting insights

---
<a href=#six></a>
## **Modeling**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Develop and train predictive or statistical models.
* **Details:** Describe the choice of models, feature selection and engineering processes, and show how the models are trained. Include code for setting up the models and explanations of the model parameters.
---


In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a href=#seven></a>
## **Evaluation and Validation**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Evaluate and validate the effectiveness and accuracy of the models.
* **Details:** Present metrics used to evaluate the models, such as accuracy, precision, recall, F1-score, etc. Discuss validation techniques employed, such as cross-validation or train/test split.
---

In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a href=#eight></a>
## **Final Model**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Present the final model and its performance.
* **Details:** Highlight the best-performing model and discuss its configuration, performance, and why it was chosen over others.
---


In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a href=#nine></a>
## **Conclusion and Future Work**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Summarize the findings and discuss future directions.
* **Details:** Conclude with a summary of the results, insights gained, limitations of the study, and suggestions for future projects or improvements in methodology or data collection.
---


In [ ]:
#Please use code cells to code in and do not forget to comment your code.

---
<a href=#ten></a>
## **References**
<a href=#cont>Back to Table of Contents</a>

* **Purpose:** Provide citations and sources of external content.
* **Details:** List all the references and sources consulted during the project, including data sources, research papers, and documentation for tools and libraries used.
---

In [ ]:
#Please use code cells to code in and do not forget to comment your code.

## Additional Sections to Consider

* ### Appendix: 
For any additional code, detailed tables, or extended data visualizations that are supplementary to the main content.

* ### Contributors: 
If this is a group project, list the contributors and their roles or contributions to the project.
